# ColabAlign

### Fast pairwise protein secondary structure comparisons using multiprocessing

[![DOI](https://zenodo.org/badge/788453062.svg)](https://doi.org/10.5281/zenodo.14169501)

Create a phylogenetic tree that compares the secondary structure of proteins, rather than nucleotide or amino acid sequence. Scoring uses the [_US-align_](https://zhanggroup.org/US-align/) algorithm by [_Zhang et al. (2022)_](https://doi.org/10.1038/s41592-022-01585-1). Based on [_mTM-align_](http://yanglab.nankai.edu.cn/mTM-align/) by [_Dong et al. (2018)_](https://doi.org/10.1093/nar/gky430). Made possible with [US-align](https://github.com/pylelab/USalign) and [Biopython](https://biopython.org).

A score of **<0.17** indicates similarity indistinguishable from a random pair of structures, where as as score **≥0.50** indicates a pair with broadly the same fold ([_Xu et al., 2010_](https://doi.org/10.1093/bioinformatics/btq066))

##### Usage

* For the best performance, click `Runtime` -> `Change runtime type` -> `TPU v2`

* If `TPU v2` is not available, `L4 GPU` or `T4 GPU` runtimes (paid options) are the next best options. These perform about 10x faster than the default `CPU` runtime. I recommend using `TPU v2`, `L4 GPU` or `T4 GPU` for datasets containing > 250 structures.

* Upload .pdb or .cif format files directly to the Colab instance by clicking the folder icon on the left, then dragging and dropping your structures.

* US-align only considers the first chain in each .pdb or .cif file, so please ensure this is the chain you wish to include in the pairwise alignment.

* Phylogenetic trees generated by ColabAlign should be viewed/analysed as un-rooted trees.

In [1]:
# @title Download and Compile the latest version of US-align, Biopython and pandas

import os
import ipywidgets as widgets
from IPython.display import display

threshold = 0.1 # @param {type:"slider", min:0.01, max:1.00, step:0.01}

os.environ['threshold'] = str(threshold)

!pip install biopython matplotlib treecluster pandas numpy

# Download and compile an up-to-date copy of US-align
!git clone https://github.com/pylelab/USalign.git USalign_git
!g++ -static -O3 -ffast-math -lm -o USalign USalign_git/USalign.cpp

!wget https://raw.githubusercontent.com/crfield18/ColabAlign/refs/heads/main/colabalign.py -O colabalign.py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.9 MB/s eta 0:00:00
Cloning into 'USalign_git'...
remote: Enumerating objects: 1542, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 1542 (delta 381), reused 369 (delta 352), pack-reused 1135 (from 2)
Receiving objects: 100% (1542/1542), 798.69 KiB | 6.19 MiB/s, done.
Resolving deltas: 100% (1103/1103), done.
--2025-02-26 12:44:06--  https://raw.githubusercontent.com/crfield18/ColabAlign/refs/heads/main/colabalign.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21351 (21K) [text/plain]
Saving to: ‘colabalign.py’

colabalign.py       100%[===================>]  20.85K  --.-KB/s    in 0.002s  

2025-02-26 12:44:06 (12.5 MB/s) - ‘cola

In [ ]:
# @title Align Structures

# @markdown Make sure your .pdb and/or .cif files are uploaded before running this block.

%%script bash

mkdir -p results models

tar -xzf *tar.gz > /dev/null 2>&1

# Delete any PDB or mmCIF files downloaded from the US-align repository.
# US-align comes with 2 example PDB files and we don't want to add these to our tree.
rm USalign_git/*.pdb

# Move all pdb and cif files to the models directory to make file handling with colabalign.py a lot easier
find . -type f \( -name "*.pdb" -o -name "*.cif" \) | tee /content/results/models.txt | xargs -I {} mv {} models/ > /dev/null 2>&1

# Count the number of models in models/ to show how many alignments are needed.
# Mainly to estimate how long the job will take.
model_count=$(find models/ -type f | wc -l)
calc_count=$((($model_count * ($model_count - 1)) / 2))
echo "There are ${model_count} files in the models directory. This means ${calc_count} total calculations."

# Run colabalign.py with all available cores for the current runtype.
python3 colabalign.py -i /content/models -o /content/results -c $(nproc --all) -t ${threshold} -p /content/USalign


In [ ]:
# @title Zip and download results

import os
import datetime
from google.colab import files

# Name the zipped results file using the current date and time
# to not accidentally overwrite older results files when downloading
current_dt = datetime.datetime.now()
zip_filename = f'colabalign_results_{current_dt.strftime("%Y%m%d-%H%M")}.zip'

# Using the built-in zip function rather than a python module for improved efficiency
os.system(f'zip -r {zip_filename} results')
files.download(zip_filename)
